# Data Collection

## Import the ImageDataGenerator library

In [1]:
from keras.preprocessing.image import ImageDataGenerator

/Users/aravind/opt/anaconda3/envs/mlp/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Configure ImageDataGenerator class

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

## Apply ImageDataGenerator functionality to Trainset and Testset

In [3]:
x_train = train_datagen.flow_from_directory('../../Dataset/train/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('../../Dataset/test/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


# Model building

## Importing the Model Building Libraries

In [4]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


## Initializing the Model

In [5]:
classifier = Sequential()

### Adding CNN Layers

In [6]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1) ,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

### Adding Dense Layers

In [7]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))

In [8]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

## Configure The Learning Process

In [9]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
classifier.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=20, validation_data=x_test, validation_steps=len(x_test)
)

Epoch 1/20


<ipython-input-11-b43cdd6e95be>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


119/119 [==============================] - 2s 15ms/step - loss: 0.1782 - accuracy: 0.9444 - val_loss: 0.3841 - val_accuracy: 0.8000
Epoch 2/20
119/119 [==============================] - 2s 15ms/step - loss: 0.1146 - accuracy: 0.9630 - val_loss: 0.2360 - val_accuracy: 0.9333
Epoch 3/20
119/119 [==============================] - 2s 14ms/step - loss: 0.0902 - accuracy: 0.9646 - val_loss: 0.2375 - val_accuracy: 0.9333
Epoch 4/20
119/119 [==============================] - 2s 17ms/step - loss: 0.0734 - accuracy: 0.9731 - val_loss: 0.1519 - val_accuracy: 0.9333
Epoch 5/20
119/119 [==============================] - 2s 16ms/step - loss: 0.0676 - accuracy: 0.9798 - val_loss: 0.2400 - val_accuracy: 0.9667
Epoch 6/20
119/119 [==============================] - 2s 15ms/step - loss: 0.0690 - accuracy: 0.9781 - val_loss: 0.3626 - val_accuracy: 0.9333
Epoch 7/20
119/119 [==============================] - 2s 15ms/step - loss: 0.0355 - accuracy: 0.9848 - val_loss: 0.2667 - val_accuracy: 0.9667
Epoch 8/20

## Save the Model

In [12]:
classifier.save('gesture.h5')

In [13]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as f:
    f.write(model_json)

## Test the model

In [14]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image

In [15]:
model = load_model("gesture.h5")

In [17]:
from tensorflow.keras.utils import load_img, img_to_array

img = load_img("../../Dataset/test/0/0.jpg", grayscale=True, target_size=(64, 64))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)

pred

/Users/aravind/opt/anaconda3/envs/mlp/lib/python3.8/site-packages/keras/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


1/1 [==============================] - 0s 199ms/step


array([[1., 0., 0., 0., 0., 0.]], dtype=float32)

In [18]:
pred[0]

array([1., 0., 0., 0., 0., 0.], dtype=float32)